In [1]:
import pandas as pd
import os
from sklearn.model_selection import KFold

#### Hold out one hospital each time and plit the opther 3 into 5 fold CV for training Save to a splits folder and thene repete for the other 3 centers...




In [2]:

# Load your main CSV file (replace with the actual file path)
main_df = pd.read_csv('/mnt/ncshare/ozkilim/BRCA/data/tasks/HGSOC_multimodal_primary_protein_and_phospho.csv')

# Focus on the 'Sample source' and 'slide_id' columns
data = main_df[['Sample Source', 'slide_id']]

# Get unique sample sources
sample_sources = data['Sample Source'].unique()

# Create a KFold object for 5 folds
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Directory to store split files
split_dir = 'split_files'
os.makedirs(split_dir, exist_ok=True)

# Iterate over each sample source to create split files
for test_source in sample_sources:
    # Filter data for cross-validation and test sets
    cv_data = data[data['Sample Source'] != test_source]['slide_id']
    test_data = data[data['Sample Source'] == test_source]['slide_id']

    # Apply 5-fold cross-validation
    for fold, (train_index, val_index) in enumerate(kf.split(cv_data)):
        # Extract train, val, and test slide IDs
        train_ids = cv_data.iloc[train_index].tolist()
        val_ids = cv_data.iloc[val_index].tolist()
        test_ids = test_data.tolist()

        # Determine the maximum length among train, val, and test lists
        max_len = max(len(train_ids), len(val_ids), len(test_ids))

        # Extend lists to have the same length
        train_ids.extend([""] * (max_len - len(train_ids)))
        val_ids.extend([""] * (max_len - len(val_ids)))
        test_ids.extend([""] * (max_len - len(test_ids)))

        # Create the split DataFrame
        split_df = pd.DataFrame({'train': train_ids[:max_len], 'val': val_ids[:max_len], 'test': test_ids[:max_len]})

        # Save the split file
        split_filename = f'/mnt/ncshare/ozkilim/BRCA/splits/{test_source}_primary/splits_{fold}.csv'
        # split_filepath = os.path.join(split_dir, split_filename)
        split_df.to_csv(split_filename, index=True)

print("Split files created successfully.")

Split files created successfully.
